In [154]:
# Importing required libraries 
import math
import pandas as pd

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import h5py
from joblib import dump
from joblib import load

from sklearn.metrics import r2_score
from scipy import stats
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, accuracy_score, r2_score, roc_auc_score, precision_score, recall_score, f1_score

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.models import load_model

sns.set()

In [155]:
loaded_model = load(r'D:\University Work\FYP\Seleted Projects\Predictive Maintanance\Code Files\Code\random_forest_model.joblib')

In [156]:

def RUL_calculator(df, df_max_cycles):
    max_cycle = df_max_cycles["cycle"]
    result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='id', right_index=True)
    result_frame["RUL"] = result_frame["max_cycle"] - result_frame["cycle"] 
    result_frame.drop(['max_cycle'], axis=1, inplace=True)
    return result_frame


In [157]:

def preprocess_data(input_data):
    # Drop irrelevant columns, including sensor22 and sensor23
    input_data.drop(['cycle', 'op1', 'op2', 'op3', 'sensor1', 'sensor5', 'sensor6', 'sensor10', 
                     'sensor16', 'sensor18', 'sensor19', 'sensor14', 'sensor13', 'sensor12', 
                     'sensor11', 'sensor9', 'sensor22', 'sensor23'], axis=1, inplace=True)
    
    # Scale the features using MinMaxScaler
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(input_data.drop(['id','RUL'], axis=1))
    scaled_features = pd.DataFrame(scaled_features, columns=input_data.drop(['id','RUL'], axis=1).columns)
    
    # Add the 'id' and 'RUL' columns back to the scaled features
    scaled_features['id'] = input_data['id']
    scaled_features['RUL'] = input_data['RUL']
    
    return scaled_features




In [158]:
def predict_remaining_useful_life(input_data):
    # Preprocess the input data
    preprocessed_data = preprocess_data(input_data)
    
    # Remove the RUL feature before prediction
    preprocessed_data.drop('RUL', axis=1, inplace=True)
    
    # Use the trained model to predict labels
    predicted_labels = loaded_model.predict(preprocessed_data)
    
    # Create a DataFrame with engine ID, predicted class, and actual RUL
    result_df = pd.DataFrame({'Engine_ID': input_data['id'], 
                              'Predicted_Class': predicted_labels,
                              'Actual_RUL': jet_data['RUL']})
    
    # Group the DataFrame by engine ID and return
    result_grouped = result_df.groupby('Engine_ID').first().reset_index()
    
    return result_grouped


In [159]:

import csv
import os

def fill_csv_with_user_input(columns, directory, filename='user_jet_data_input.csv'):
    filepath = os.path.join(directory, filename)
    with open(filepath, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(columns)

    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        while True:
            row = []
            for col in columns:
                value = input(f"Enter value for {col}: ")
                row.append(value)
            writer.writerow(row)
            another_entry = input("Do you want to enter another record? (yes/no): ")
            if another_entry.lower() != 'yes':
                break

    print(f"CSV file '{filename}' filled with user input successfully in directory '{directory}'.")


In [160]:

# Example usage:
columns = ["id","cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5"
           ,"sensor6","sensor7","sensor8","sensor9","sensor10","sensor11","sensor12","sensor13"
           ,"sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
           ,"sensor20","sensor21","sensor22","sensor23"]

directory = r'D:\University Work\FYP\Seleted Projects\Predictive Maintanance\Code Files\Code'

fill_csv_with_user_input(columns,directory)

user_input_data = pd.read_csv(r'D:\University Work\FYP\Seleted Projects\Predictive Maintanance\Code Files\Code\user_jet_data_input.csv')  # Modify this line as needed

jet_id_and_rul = user_input_data.groupby(['id'])[["id" ,"cycle"]].max()
jet_id_and_rul.set_index('id', inplace=True)
jet_data = RUL_calculator(user_input_data, jet_id_and_rul)


In [161]:

    
# Predict the remaining useful life (RUL) using the user input data
predicted_RUL = predict_remaining_useful_life(jet_data)
    
# Display or use the predicted RUL as needed
print("Predicted Remaining Useful Life (RUL) for the input data:")
predicted_RUL

Predicted Remaining Useful Life (RUL) for the input data:


,Engine_ID,Predicted_Class,Actual_RUL
0,57,0,132
1,101,0,0
